# This analysis process for sentiment analysis

In [1]:
import timeit

from gensim.models import word2vec
import numpy as np

from sentiment.utilities import dataset_split
from sentiment.utilities import load_data
from sentiment.utilities import text2vec
from sentiment.utilities import y_trainable

data = load_data()

# Preprocess and convert words and bigrams to integer IDs
# We're using the 20000 most frequent words and 20000 most frequent bigrams
# from the IMDB dataset here: http://ai.stanford.edu/~amaas/data/sentiment/
X = text2vec(data.text)
y = data.ratings.values * 10

z = data.reviewers.values

idc_all = dataset_split(y, holdout=0.3, validation=0.0)

X_train = X[idc_all[0]]
y_train_origin = y[idc_all[0]]
y_train = y_trainable(y_train_origin)

X_val = X[idc_all[1]]
y_val_origin = y[idc_all[1]]
y_val = y_trainable(y_val_origin)

data_test = load_data(category='test')
X_test = text2vec(data_test.text)
y_test_origin = data_test.ratings.values * 10

Using TensorFlow backend.


## Classifier 1:SVM(Support Vector Machine)

In [2]:
from sklearn.svm import SVC
from sentiment.utilities import mean_absolute_error

clf_1 = SVC(C = 2, probability = True)
clf_1.fit(X_train, y_train)

SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [3]:
y_train_predicted = clf_1.predict(X_train)
mean_absolute_error(y_train_origin, y_train_predicted)

0.14589999999999997

In [4]:
y_val_predicted = clf_1.predict(X_val)
mean_absolute_error(y_val_origin, y_val_predicted)

0.14337662337662338

In [5]:
y_test_predicted = clf_1.predict(X_test)
mean_absolute_error(y_test_origin, y_test_predicted)

0.15410199556541021

## Classifier 2:Naive Bayes

In [6]:
from sklearn.naive_bayes import GaussianNB

clf_2 = GaussianNB()
clf_2.fit(X_train, y_train)

GaussianNB(priors=None)

In [7]:
y_train_predicted = clf_2.predict(X_train)
mean_absolute_error(y_train_origin, y_train_predicted)

0.22409285714285715

In [8]:
y_val_predicted = clf_2.predict(X_val)
mean_absolute_error(y_val_origin, y_val_predicted)

0.2257316017316017

In [9]:
y_test_predicted = clf_2.predict(X_test)
mean_absolute_error(y_test_origin, y_test_predicted)

0.24223946784922396

## Classifier 3:Random Forest

In [10]:
from sklearn.ensemble import RandomForestClassifier

clf_3 = RandomForestClassifier(min_samples_leaf = 3, n_estimators = 100)
clf_3.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=3,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [11]:
y_train_predicted = clf_3.predict(X_train)
mean_absolute_error(y_train_origin, y_train_predicted)

0.0063785714285714279

In [12]:
y_val_predicted = clf_3.predict(X_val)
mean_absolute_error(y_val_origin, y_val_predicted)

0.13045887445887444

In [13]:
y_test_predicted = clf_3.predict(X_test)
mean_absolute_error(y_test_origin, y_test_predicted)

0.16363636363636364

## Classifier 4:BernoulliRBM

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import BernoulliRBM

rbm = BernoulliRBM(random_state = 0, verbose = True)
logistic = LogisticRegression()

rbm.learning_rate = 0.07
rbm.n_iter = 50
# more components tend to give better prediction performance, but larger fitting time
rbm.n_components = 800
rbm.batch_size = 10
logistic.C = 10000.0

clf_4 = Pipeline(steps = [('rbm', rbm), ('logistic', logistic)])
clf_4.fit(X_train, y_train)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -22.14, time = 0.55s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -25.52, time = 0.74s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -29.67, time = 0.75s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -19.14, time = 0.74s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -27.89, time = 0.73s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -22.97, time = 0.75s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -34.20, time = 0.87s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -17.95, time = 0.79s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -15.79, time = 0.74s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -18.57, time = 0.66s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -27.72, time = 0.67s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -30.97, time = 0.66s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -32.23, time = 0.66s
[BernoulliRBM] Iteration 14, pseudo-likelihood = -19.77, time = 0.67s
[BernoulliRBM] Iteration 15, 

Pipeline(steps=[('rbm', BernoulliRBM(batch_size=10, learning_rate=0.07, n_components=800, n_iter=50,
       random_state=0, verbose=True)), ('logistic', LogisticRegression(C=10000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))])

In [15]:
y_train_predicted = clf_4.predict(X_train)
mean_absolute_error(y_train_origin, y_train_predicted)

0.14632857142857142

In [16]:
y_val_predicted = clf_4.predict(X_val)
mean_absolute_error(y_val_origin, y_val_predicted)

0.14346320346320349

In [17]:
y_test_predicted = clf_4.predict(X_test)
mean_absolute_error(y_test_origin, y_test_predicted)

0.15410199556541021